In [3]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [4]:
# Carrega arquivo parquet
df = pd.read_parquet("DB/List-dependencies.parquet")

In [5]:
dfs = {}
graphs = {}
dfs["Geral"] = df
graphs["Geral"] = nx.from_pandas_edgelist(df, source="Project Name", target="Dependency Name", create_using=nx.DiGraph())
for platform in df["Platform"].unique():
    # cria um dataframe para cada plataforma "df["Platform"].unique()" e adiciona a lista de dataframes ao dicionário "dfs"
    dfs[platform] = df[df["Platform"] == platform]

    # Cria um grafo direcionado
    graphs[platform] = nx.from_pandas_edgelist(dfs[platform], source="Project Name", target="Dependency Name", create_using=nx.DiGraph())

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import time
from tqdm.notebook import tqdm
from networkx.algorithms import community

def format_report(graph_name, metrics):
    """Formata as métricas coletadas em uma string Markdown."""
    report = [f"# Análise do Grafo: {graph_name}\n"]
    
    # Métricas Básicas
    report.append("## 1. Métricas Gerais")
    report.append("| Métrica | Valor |")
    report.append("|---|---|")
    report.append(f"| **Nós (Projetos/Dependências)** | {metrics['nodes']:,} |")
    report.append(f"| **Arestas (Dependências)** | {metrics['edges']:,} |")
    report.append(f"| **Densidade do Grafo** | {metrics['density']:.8f} |")
    report.append(f"| **Grafo Direcionado** | {'Sim' if metrics['is_directed'] else 'Não'} |")
    
    # Conectividade
    report.append("\n## 2. Análise de Conectividade")
    report.append("| Métrica | Valor |")
    report.append("|---|---|")
    report.append(f"| **Componentes Fracamente Conectados** | {metrics['wcc_count']:,} |")
    report.append(f"| **Nós no Maior Componente (LCC)** | {metrics['lcc_nodes']:,} ({metrics['lcc_percent']:.2f}%) |")
    report.append(f"| **Arestas no Maior Componente (LCC)** | {metrics['lcc_edges']:,} ({metrics['lcc_edges_percent']:.2f}%) |")

    # Análise de Graus
    report.append("\n## 3. Análise de Graus (Popularidade e Complexidade)")
    report.append("### 3.1. Estatísticas Gerais de Grau")
    report.append("| Métrica | In-Degree (Popularidade) | Out-Degree (Complexidade) | Grau Total |")
    report.append("|---|---|---|---|")
    report.append(f"| **Média** | {metrics['in_degree_mean']:.2f} | {metrics['out_degree_mean']:.2f} | {metrics['degree_mean']:.2f} |")
    report.append(f"| **Mediana** | {metrics['in_degree_median']:.2f} | {metrics['out_degree_median']:.2f} | {metrics['degree_median']:.2f} |")
    report.append(f"| **Máximo** | {metrics['in_degree_max']:,} | {metrics['out_degree_max']:,} | {metrics['degree_max']:,} |")

    report.append("\n### 3.2. Top 10 Dependências Mais Populares (Maior In-Degree)")
    report.append("| Rank | Dependência | In-Degree |")
    report.append("|---|---|---|")
    for i, (node, degree) in enumerate(metrics['top_in_degree'], 1):
        report.append(f"| {i} | `{node}` | {degree:,} |")

    report.append("\n### 3.3. Top 10 Projetos Mais Complexos (Maior Out-Degree)")
    report.append("| Rank | Projeto | Out-Degree |")
    report.append("|---|---|---|")
    for i, (node, degree) in enumerate(metrics['top_out_degree'], 1):
        report.append(f"| {i} | `{node}` | {degree:,} |")

    # Centralidade
    if 'pagerank' in metrics:
        report.append("\n## 4. Análise de Centralidade")
        report.append(f"_(Análise realizada em {metrics.get('centrality_info', 'todo o grafo')})_")
        report.append("\n### 4.1. Top 10 Nós por Influência (PageRank)")
        report.append("| Rank | Nó | PageRank Score |")
        report.append("|---|---|---|")
        for i, (node, score) in enumerate(metrics['pagerank'], 1):
            report.append(f"| {i} | `{node}` | {score:.6f} |")
        
        if 'betweenness' in metrics:
            report.append("\n### 4.2. Top 10 Nós por Importância Estrutural (Betweenness Centrality)")
            report.append("| Rank | Nó | Betweenness Score |")
            report.append("|---|---|---|")
            for i, (node, score) in enumerate(metrics['betweenness'], 1):
                report.append(f"| {i} | `{node}` | {score:.6f} |")

    # Análise Estrutural
    if 'avg_clustering' in metrics:
        report.append("\n## 5. Análise Estrutural e de Comunidades")
        report.append(f"| Métrica | Valor |")
        report.append(f"|---|---|")
        report.append(f"| **Coeficiente de Clusterização Médio** | {metrics['avg_clustering']:.6f} |")
        if 'communities' in metrics:
            report.append(f"| **Comunidades Detectadas (Louvain)** | {metrics['communities']:,} |")
            report.append(f"| **Modularidade** | {metrics['modularity']:.4f} |")

    report.append(f"\n---\n*Análise concluída em {metrics['elapsed_time']:.2f} segundos.*\n\n")
    return "\n".join(report)


def analyze_graph(graph, name, is_large_scale=False):
    """Função principal para analisar um grafo e coletar métricas."""
    print(f"Iniciando análise para o grafo: {name}...")
    start_time = time.time()
    metrics = {}

    # 1. Métricas Básicas
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    metrics.update({
        'nodes': num_nodes,
        'edges': num_edges,
        'density': nx.density(graph),
        'is_directed': graph.is_directed()
    })

    if num_nodes == 0:
        metrics['elapsed_time'] = time.time() - start_time
        return metrics

    # 2. Conectividade
    wcc = list(nx.weakly_connected_components(graph))
    largest_wcc_nodes = max(wcc, key=len)
    lcc = graph.subgraph(largest_wcc_nodes)
    metrics.update({
        'wcc_count': len(wcc),
        'lcc_nodes': lcc.number_of_nodes(),
        'lcc_percent': (lcc.number_of_nodes() / num_nodes) * 100 if num_nodes > 0 else 0,
        'lcc_edges': lcc.number_of_edges(),
        'lcc_edges_percent': (lcc.number_of_edges() / num_edges) * 100 if num_edges > 0 else 0,
    })

    # 3. Análise de Grau
    in_degrees = dict(graph.in_degree())
    out_degrees = dict(graph.out_degree())
    degrees = dict(graph.degree())
    metrics.update({
        'in_degree_mean': np.mean(list(in_degrees.values())),
        'in_degree_median': np.median(list(in_degrees.values())),
        'in_degree_max': max(in_degrees.values() or [0]),
        'out_degree_mean': np.mean(list(out_degrees.values())),
        'out_degree_median': np.median(list(out_degrees.values())),
        'out_degree_max': max(out_degrees.values() or [0]),
        'degree_mean': np.mean(list(degrees.values())),
        'degree_median': np.median(list(degrees.values())),
        'degree_max': max(degrees.values() or [0]),
        'top_in_degree': sorted(in_degrees.items(), key=lambda item: item[1], reverse=True)[:10],
        'top_out_degree': sorted(out_degrees.items(), key=lambda item: item[1], reverse=True)[:10]
    })

    # 4. Centralidade (Otimizado para escala)
    target_graph_for_centrality = lcc
    if is_large_scale:
        # Para grafos muito grandes, amostrar é mais seguro mesmo com muita RAM
        sample_size = min(num_nodes, 200000) # Amostra grande de 200k nós
        sample_nodes = np.random.choice(list(lcc.nodes()), sample_size, replace=False)
        target_graph_for_centrality = lcc.subgraph(sample_nodes)
        metrics['centrality_info'] = f"uma amostra de {sample_size:,} nós do maior componente"
    
    print(f"  Calculando PageRank em {target_graph_for_centrality.number_of_nodes():,} nós...")
    pagerank = nx.pagerank(target_graph_for_centrality, alpha=0.85)
    metrics['pagerank'] = sorted(pagerank.items(), key=lambda item: item[1], reverse=True)[:10]

    # Betweenness é muito pesado. Para o grafo grande, usamos uma aproximação com k.
    # Para os menores, calculamos no LCC inteiro.
    if is_large_scale:
        k_betweenness = min(1000, target_graph_for_centrality.number_of_nodes())
        print(f"  Calculando Betweenness (aproximado com k={k_betweenness})...")
        betweenness = nx.betweenness_centrality(target_graph_for_centrality, k=k_betweenness, normalized=True)
        metrics['centrality_info'] += f" (Betweenness aproximado com k={k_betweenness})"
    else:
        print(f"  Calculando Betweenness em {target_graph_for_centrality.number_of_nodes():,} nós...")
        betweenness = nx.betweenness_centrality(target_graph_for_centrality, normalized=True)
    metrics['betweenness'] = sorted(betweenness.items(), key=lambda item: item[1], reverse=True)[:10]

    # 5. Análise Estrutural (Apenas para grafos menores, pois são caros)
    if not is_large_scale:
        print("  Calculando Coeficiente de Clusterização...")
        metrics['avg_clustering'] = nx.average_clustering(graph)
        
        print("  Detectando comunidades (Louvain)...")
        # O algoritmo de comunidade funciona melhor em grafos não direcionados
        communities_sets = community.louvain_communities(graph.to_undirected(), seed=42)
        modularity = community.modularity(graph.to_undirected(), communities_sets)
        metrics.update({
            'communities': len(communities_sets),
            'modularity': modularity
        })

    metrics['elapsed_time'] = time.time() - start_time
    print(f"Análise de {name} concluída em {metrics['elapsed_time']:.2f} segundos.\n")
    return metrics

# --- Bloco Principal de Execução ---
final_report_md = []

# Analisa o grafo "Geral" com a estratégia para larga escala
if "Geral" in graphs:
    geral_metrics = analyze_graph(graphs["Geral"], "Geral", is_large_scale=True)
    final_report_md.append(format_report("Geral", geral_metrics))

# Analisa os outros grafos com a estratégia padrão
sorted_platforms = sorted([p for p in graphs.keys() if p != "Geral"])
for platform_name in tqdm(sorted_platforms, desc="Analisando Plataformas"):
    platform_graph = graphs[platform_name]
    platform_metrics = analyze_graph(platform_graph, platform_name, is_large_scale=False)
    final_report_md.append(format_report(platform_name, platform_metrics))

# Salva o relatório final em um arquivo .md
output_filename = "analise_grafos.md"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write("\n".join(final_report_md))

print(f"Relatório completo salvo em: {output_filename}")

Iniciando análise para o grafo: Geral...
  Calculando PageRank em 200,000 nós...
  Calculando Betweenness (aproximado com k=1000)...
Análise de Geral concluída em 1632.50 segundos.



Analisando Plataformas:   0%|          | 0/18 [00:00<?, ?it/s]

Iniciando análise para o grafo: Atom...
  Calculando PageRank em 6,770 nós...
  Calculando Betweenness em 6,770 nós...
  Calculando Coeficiente de Clusterização...
  Detectando comunidades (Louvain)...
Análise de Atom concluída em 293.65 segundos.

Iniciando análise para o grafo: CPAN...
  Calculando PageRank em 32,898 nós...
  Calculando Betweenness em 32,898 nós...
  Calculando Coeficiente de Clusterização...
  Detectando comunidades (Louvain)...
Análise de CPAN concluída em 7938.59 segundos.

Iniciando análise para o grafo: CRAN...
  Calculando PageRank em 15,409 nós...
  Calculando Betweenness em 15,409 nós...
  Calculando Coeficiente de Clusterização...
  Detectando comunidades (Louvain)...
Análise de CRAN concluída em 1819.06 segundos.

Iniciando análise para o grafo: Cargo...
  Calculando PageRank em 23,951 nós...
  Calculando Betweenness em 23,951 nós...
  Calculando Coeficiente de Clusterização...
  Detectando comunidades (Louvain)...
Análise de Cargo concluída em 4051.54 segu